In [1]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers


# Load the dataset
data = pd.read_csv("combined_data.csv")

data.head()

,_id,Initial_Year,Initial_Month,Forecast_Year,Foreast_Month,Province_ID,Province_NameTH,Rainfall(mm),Rainfall.mm.
0,1.0,2021,1,2021,1,10,กรุงเทพมหานคร,5.359580,NaN
1,2.0,2021,1,2021,1,11,จ.สมุทรปราการ,4.724574,NaN
2,3.0,2021,1,2021,1,12,จ.นนทบุรี,6.060826,NaN
3,4.0,2021,1,2021,1,13,จ.ปทุมธานี,6.709093,NaN
4,5.0,2021,1,2021,1,14,จ.พระนครศรีอยุธยา,2.295685,NaN


In [3]:
# 2. ทำการแปลงข้อมูล
data['Rainfall(mm)'] = (data['Rainfall(mm)'] > 0).astype(int)  # แปลง Rainfall ให้เป็น 0/1

# 3. แยกฟีเจอร์และเป้าหมาย
X = data[['Initial_Year', 'Initial_Month', 'Forecast_Year', 'Foreast_Month', 'Province_ID']]
y = data['Rainfall(mm)']

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. ทำการ normalize ข้อมูล
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5. สร้างโมเดล Deep Learning
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # ใช้ sigmoid สำหรับการจำแนกประเภท
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. ฝึกโมเดล
model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1)

# 7. ประเมินโมเดล
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

model.save('rain_prediction_model.h5')
print("โมเดลถูกบันทึกเรียบร้อยแล้ว")

# ฟังก์ชันเพื่อคาดการณ์ความน่าจะเป็นฝน
def predict_rain_probability(model, scaler, input_data):
    input_data_scaled = scaler.transform([input_data])
    probability = model.predict(input_data_scaled)[0][0] * 100  # แปลงเป็นเปอร์เซ็นต์
    return probability

# ตัวอย่างการใช้ฟังก์ชัน
example_input = [2021, 1, 2024, 9, 9]  # ปี, เดือนเริ่ม, ปีคาดการณ์, เดือนคาดการณ์, Province_ID
rain_probability = predict_rain_probability(model, scaler, example_input)
print(f'ความน่าจะเป็นว่าฝนจะตก: {rain_probability:.2f}%')

Epoch 1/50


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1220/1220 ━━━━━━━━━━━━━━━━━━━━ 2s 894us/step - accuracy: 0.9200 - loss: 0.2277
Epoch 2/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 826us/step - accuracy: 0.9845 - loss: 0.0591
Epoch 3/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 817us/step - accuracy: 0.9850 - loss: 0.0527
Epoch 4/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.9870 - loss: 0.0446
Epoch 5/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 753us/step - accuracy: 0.9862 - loss: 0.0414
Epoch 6/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 711us/step - accuracy: 0.9857 - loss: 0.0431
Epoch 7/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 816us/step - accuracy: 0.9864 - loss: 0.0409
Epoch 8/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 699us/step - accuracy: 0.9855 - loss: 0.0446
Epoch 9/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 725us/step - accuracy: 0.9859 - loss: 0.0378
Epoch 10/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 753us/step - accuracy: 0.9863 - loss: 0.0393
Epoch 11/50
1220/1220 ━━━━━━━━━━━━━━━━━━━━ 1s 769us/step - accuracy: 0.9856 - loss: 0.0365
Epoch 12/50
1220/12

Loss: 0.02555469237267971, Accuracy: 0.9914754033088684
โมเดลถูกบันทึกเรียบร้อยแล้ว
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
ความน่าจะเป็นว่าฝนจะตก: 100.00%


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
